In [11]:
customers_df.head(), transactions_df.head()

(   customer_id  age  gender    country account_type  account_balance  \
 0            1   21  Female   Malaysia      Current          3365.70   
 1            2   75  Female  Indonesia      Savings         47878.37   
 2            3   51  Female      Yemen      Savings         38483.48   
 3            4   29    Male        UAE      Savings         30176.33   
 4            5   27  Female   Malaysia      Savings         22681.96   
 
    credit_score   join_date  
 0           671  2022-01-03  
 1           514  2018-11-01  
 2           761  2024-06-30  
 3           304  2024-01-07  
 4           497  2025-01-29  ,
    transaction_id  customer_id           transaction_date   amount  \
 0           50000          950 2025-07-28 13:59:41.558580  2358.27   
 
   transaction_type channel merchant_category  is_fraud  hour  amount_log  \
 0          Payment     POS         Groceries         0    13    7.766108   
 
    is_night  
 0     False  )

In [ ]:
!pip install sdv faker pandas numpy

In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random

# تهيئة Faker
fake = Faker()

# عدد العملاء
N_CUSTOMERS = 5000
customers = []

for i in range(1, N_CUSTOMERS + 1):
  customers.append({
  "customer_id": i,
  "age": random.randint(18, 75),
  "gender": random.choice(["Male", "Female"]),
  "country": random.choice(["Yemen", "Indonesia", "Malaysia", "UAE"]),
  "account_type": random.choice(["Savings", "Current"]),
  "account_balance": round(random.uniform(500, 50000), 2),
  "credit_score": random.randint(300, 850),
  "join_date": fake.date_between(start_date="-10y", end_date="today")
  })

customers_df = pd.DataFrame(customers)
customers_df.head()


,customer_id,age,gender,country,account_type,account_balance,credit_score,join_date
0,1,21,Female,Malaysia,Current,3365.70,671,2022-01-03
1,2,75,Female,Indonesia,Savings,47878.37,514,2018-11-01
2,3,51,Female,Yemen,Savings,38483.48,761,2024-06-30
3,4,29,Male,UAE,Savings,30176.33,304,2024-01-07
4,5,27,Female,Malaysia,Savings,22681.96,497,2025-01-29


In [ ]:
import numpy as np
import random

# دالة وقت المعاملة بسلوك واقعي (النهار أكثر نشاطًا)
# توزيع مضمون: مجموع الاحتمالات = 1 بدقة
def random_transaction_time():
# 00–05 (6 ساعات): هادئ جداً
# 06–15 (10 ساعات): نشط
# 16–23 (8 ساعات): متوسط
  hour_distribution = (
  [0.02] * 6 + # 0.12
  [0.06] * 10 + # 0.60
  [0.035] * 8 # 0.28
  ) # المجموع = 1.00

  hour = np.random.choice(range(24), p=hour_distribution)
  dt = fake.date_time_between(start_date="-1y", end_date="now")
  return dt.replace(hour=hour, minute=random.randint(0, 59), second=random.randint(0, 59))

  # توليد المعاملات مع منطق واقعي
N_TRANSACTIONS = 50000
transactions = []

for i in range(1, N_TRANSACTIONS + 1):
  customer = customers_df.sample(1).iloc[0]

  # ربط المبلغ بالرصيد: لا مبالغ ضخمة إذا الرصيد صغير
  max_amount = customer["account_balance"] * random.uniform(0.1, 0.9)
  amount = round(np.random.uniform(5, max_amount), 2)

# وقت المعاملة وسلوك الليل
transaction_dt = random_transaction_time()
hour = transaction_dt.hour
is_night = (hour <= 5) or (hour >= 20)

# منطق احتيال شرطي وواقعي
high_balance = customer["account_balance"] > 20000
high_amount = amount > 3000

fraud_probability = 0.01 # خط أساس ~1%
if high_amount:
  fraud_probability += 0.03
if is_night:
  fraud_probability += 0.02
if high_balance and high_amount and is_night:
  fraud_probability += 0.02 # تآزر مخاطرة

is_fraud = 1 if random.random() < fraud_probability else 0

transactions.append({
"transaction_id": i,
"customer_id": customer["customer_id"],
"transaction_date": transaction_dt,
"amount": amount,
"transaction_type": random.choice(["Transfer", "Payment", "Withdrawal"]),
"channel": random.choice(["ATM", "Online", "POS"]),
"merchant_category": random.choice(["Groceries", "Electronics", "Travel", "Healthcare", "Restaurants"]),
"is_fraud": is_fraud
})

transactions_df = pd.DataFrame(transactions)
transactions_df.head()


,transaction_id,customer_id,transaction_date,amount,transaction_type,channel,merchant_category,is_fraud
0,50000,950,2025-07-28 13:59:41.558580,2358.27,Payment,POS,Groceries,0


In [ ]:

# ميزات تحليلية للتعلم الآلي
transactions_df["hour"] = transactions_df["transaction_date"].dt.hour
transactions_df["amount_log"] = np.log1p(transactions_df["amount"])

# تحقق: متوسط المبلغ حسب الاحتيال
mean_by_fraud = transactions_df.groupby("is_fraud")["amount"].mean()
print("متوسط المبلغ حسب الاحتيال:\n", mean_by_fraud)

# تحقق: نسبة الاحتيال
fraud_ratio = transactions_df["is_fraud"].value_counts(normalize=True)
print("\nنسبة الاحتيال:\n", fraud_ratio)

# تحقق إضافي: مقارنة مبالغ الليل والنهار
transactions_df["is_night"] = (transactions_df["hour"] <= 5) | (transactions_df["hour"] >= 20)
mean_by_night = transactions_df.groupby("is_night")["amount"].mean()
print("\nمتوسط المبلغ - ليل مقابل نهار:\n", mean_by_night)


متوسط المبلغ حسب الاحتيال:
 is_fraud
0    2358.27
Name: amount, dtype: float64

نسبة الاحتيال:
 is_fraud
0    1.0
Name: proportion, dtype: float64

متوسط المبلغ - ليل مقابل نهار:
 is_night
False    2358.27
Name: amount, dtype: float64


In [ ]:
customers_df.to_csv("customers.csv", index=False)
transactions_df.to_csv("transactions.csv", index=False)


In [10]:
readme = """# Synthetic Retail Banking Transactions Dataset

## Overview
This project provides a **privacy-preserving synthetic banking dataset** designed for
machine learning, analytics, and fraud detection research.

The dataset simulates realistic customer behavior and transaction patterns
commonly found in retail banking systems, while containing **no real personal data**.

---

## Dataset Description

### Customers Table (`customers.csv`)
- `customer_id` — Unique customer identifier
- `age` — Customer age
- `gender` — Gender
- `country` — Country of residence
- `account_type` — Savings / Current
- `account_balance` — Current account balance
- `credit_score` — Creditworthiness score
- `join_date` — Account creation date

### Transactions Table (`transactions.csv`)
- `transaction_id` — Unique transaction identifier
- `customer_id` — Linked customer
- `transaction_date` — Timestamp of transaction
- `amount` — Transaction amount
- `transaction_type` — Transfer / Payment / Withdrawal
- `channel` — ATM / Online / POS
- `merchant_category` — Merchant category
- `is_fraud` — Fraud label (0 = normal, 1 = suspicious)

---

## Key Features
- Realistic behavioral patterns (day/night activity cycles)
- Statistically consistent transaction amounts
- Rare but meaningful fraud events (≈1–3%)
- Ready-to-use features for ML training
- Fully synthetic and GDPR-safe

---

## Use Cases
- Fraud Detection Model Training
- Risk Analysis & Simulation
- Data Science Education
- Privacy-Safe Prototyping

---

## Data Generation
The dataset was generated using Python with controlled probabilistic logic to
ensure statistical realism while avoiding data leakage.

Libraries used:
- NumPy
- Pandas
- Faker

---

## Disclaimer
This dataset is **entirely synthetic** and does not represent real customers,
accounts, or transactions.
"""

with open("README.md", "w", encoding="utf-8") as f:
  f.write(readme)